# Step 1: Run this cell first to import all libraries and functions

In [2]:
#=========== PARAMETERS =============#
# make meeting notes
query_sp = 'exec [MDA-DB-DEV-CLA-AE].planner.createMeetingNotes2 ?, ?, ?'
meeting_notes_columns = ['Section', 'Card Created DateTime', 'Bucket', 'Card Title', 'Card Labels', 'Date Description', 'Task Description', 'Comments', 'Assigneess']
meeting_notes_index_col = 'Section'

# send email
smtp_server = 'smtp.mater.org.au'
smtp_port = '587'

#=========== LIBRARIES ==============#
import pandas as pd
import pyodbc
from datetime import datetime

import io
import smtplib
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

#=========== LIST OF PLANS ==============#
PlansList = {'PLAN 1 NAME': '-6-Vic94GE-s5oiwg57nAsgAFOm6'}

#=========== FUNCTIONS ==============#

def exec_procedure_to_df(cs, query, meetingDate, planId, maxDays, meeting_notes_columns, meeting_notes_index_col):
    # make connection pyodbc
    conn = pyodbc.connect(cs)

    cursor = conn.cursor()
    conn.autocommit = True
    cursor.execute(query, meetingDate, planId, maxDays)

    results = cursor.fetchall()
    df = pd.DataFrame.from_records(results, columns=meeting_notes_columns, index=meeting_notes_index_col)
    conn.close()
    return df

def replace_breaks(df):
    '''
    Replace breaks '<br/>' with a new line character '\n' in a dataframe
    '''
    df = df.replace('<br/>', '\n', regex=True)
    return df

def export_excel(df):
    with io.BytesIO() as buffer:
        writer=pd.ExcelWriter(buffer)
        df.to_excel(writer)
        writer.save()
        return buffer.getvalue()

def send_dataframe(username, password, send_from, send_to, subject, body, df):
    multipart = MIMEMultipart()
    multipart['From'] = send_from
    multipart['To'] = send_to
    multipart['Subject'] = subject

    EXPORTERS = {'meeting_notes.xlsx': export_excel}

    for filename in EXPORTERS:
        attachment = MIMEApplication(EXPORTERS[filename](df))
        attachment['Content-Disposition'] = 'attachment; filename="{}"'.format(filename)
        multipart.attach(attachment)
    multipart.attach(MIMEText(body, 'html'))
    try:
        server = smtplib.SMTP('smtp.mater.org.au','587')
    except smtplib.socket.gaierror:
        return False
    server.ehlo()
    server.starttls()
    server.ehlo()
    try:
        server.login(username, password)
    except SMTPAuthenticationError:
        server.quit()
        print('Authentication FAILED. Incorrect login credentials. Check username and password.')
        return False
    
    server.sendmail(send_from, send_to, multipart.as_string())
    print('Email sent successfully')
    server.quit()
    
def run(query_sp, sql_db_username, sql_db_password, meetingDate, planId, maxDays, meeting_notes_columns, meeting_notes_index_col, username, password, send_to_email, send_from_email):
    
    # connect string
    cs = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "Authentication=ActiveDirectoryPassword;"
    "SERVER=mda-sql-aap-dev-ase.database.windows.net;"
    "DATABASE=MDA-DB-DEV-CLA-AE;"
    f"UID={sql_db_username};"
    f"PWD={sql_db_password};"
    )
    
    # exec proc
    df = exec_procedure_to_df(cs=cs, query=query_sp, meetingDate=meetingDate, planId=planId, maxDays=maxDays,
                              meeting_notes_columns=meeting_notes_columns,
                              meeting_notes_index_col=meeting_notes_index_col)

    # Clean the line breaks
    df = df.replace('<br/>', '\n', regex=True)

    # send the dataframe
    send_dataframe(username=username, password=password, send_to=send_to_email, send_from=send_from_email, df=df, subject=f'Meeting Notes for {meetingDate}', body='Please find the file attached')

# Step 2: Input Parameters

## 2.1. Input Meeting Date:

In [3]:
meetingDate=widgets.DatePicker(value=pd.to_datetime('2020-11-04'), description='Input Date: ')
display(meetingDate)

DatePicker(value=Timestamp('2020-11-04 00:00:00'), description='Input Date: ')

## 2.2. Input Period Length in Days:

In [4]:
maxDays=widgets.IntSlider(
    value=30,
    min=1,
    max=100,
    step=1,
    description='Input No of Days: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(maxDays)

IntSlider(value=30, continuous_update=False, description='Input No of Days: ', min=1)

## 2.3. Input Plan Name:

In [5]:
planId = widgets.Dropdown(
    options=[i for i in PlansList],
    value=list(PlansList.keys())[0],
    description='Input Plan Name: '
)
display(planId)

Dropdown(description='Input Plan Name: ', options=('PLAN 1 NAME',), value='PLAN 1 NAME')

## 2.4. Input SQL database username and password to connect to MDA-DB-DEV-CLA-AE on mda-sql-aap-dev-ase.database.windows.net:

In [6]:
sql_db_username = widgets.Text(
    value='Type your SQL username',
    placeholder='Type your SQL username',
    description='SQL username: ',
    disabled=False
)
sql_db_password = widgets.Text(
    value='Type your SQL password',
    placeholder='Type SQL password',
    description='SQL password: ',
    disabled=False
)
display(sql_db_username, sql_db_password)

Text(value='Type your SQL username', description='SQL username: ', placeholder='Type your SQL username')

Text(value='Type your SQL password', description='SQL password: ', placeholder='Type SQL password')

## 2.5. Input Outlook username and password (usually your username is your email @mater.org.au):

In [7]:
# Input username and password for outlook
outlook_username = widgets.Text(
    value='Type your username',
    placeholder='Type your username',
    description='Outlook username: ',
    disabled=False
)
outlook_password = widgets.Text(
    value='Type your password',
    placeholder='Type your password',
    description='Outlook password: ',
    disabled=False
)
display(outlook_username, outlook_password)

Text(value='Type your username', description='Outlook username: ', placeholder='Type your username')

Text(value='Type your password', description='Outlook password: ', placeholder='Type your password')

## 2.6. Input from/to email address

In [8]:
send_to_email = widgets.Text(
    value='Type email address',
    placeholder='Type email address',
    description='Send to: ',
    disabled=False
)
send_from_email = widgets.Text(
    value='Type email address',
    placeholder='Type email address',
    description='Send from: ',
    disabled=False
)
display(send_to_email, send_from_email)

Text(value='Type email address', description='Send to: ', placeholder='Type email address')

Text(value='Type email address', description='Send from: ', placeholder='Type email address')

# Step 3: Send an automatic email with meeting notes

In [9]:
button = widgets.Button(description="Get Email")

display(button)

def on_button_clicked(b):
    run(query_sp=query_sp, sql_db_username=sql_db_username.value, sql_db_password=sql_db_password.value, meetingDate=meetingDate.value.strftime("%Y-%m-%d"), planId=PlansList[planId.value], maxDays=maxDays.value, meeting_notes_columns=meeting_notes_columns, meeting_notes_index_col=meeting_notes_index_col, username=outlook_username.value, password=outlook_password.value, send_to_email=send_to_email.value, send_from_email=send_from_email.value)

button.on_click(on_button_clicked)

Button(description='Get Email', style=ButtonStyle())

# END